In [63]:
import build_train 
import pickle
import numpy
import tensorflow as tf
import parameters
max_len = 10

In [66]:
def create_sent_corpus():
    message_list = []
    with open(build_train.train_file,"r") as fid:
            c = 0
            for line in fid:
                c+=1
    #             print("len", len(line))
                if len(line) > 1:
                    message_list.append(" ".join(line.split()[1:]))
            max_len = len(max(message_list, key=len))
            
            return  message_list

In [67]:
def genTrainExamples(message_list, max_len, wrd2idx):
    feature_list = []
    #found max_len greater that embedding size don't understand a thing about it. Keeping it 10 temporarily.
    
    sent_list = []
    for sent in message_list:       
        for w in sent.split(" "):                      
            if w in wrd2idx: #don't know if we have to take words comming only in wrd2idx
                sent_list.append(wrd2idx[w])
        while len(sent_list) < max_len:
            sent_list.append(-1)
        feature_list.append(sent_list)
        sent_list = []            
    return feature_list
                

In [208]:
class CNNModel(object):
     

    def __init__(self, graph, E):

        
        self.build_graph(graph, E)

    def build_graph(self, graph, E):
        """

        :param graph:
        :param embedding_array:
        :param Config:
        :return:
        """
        pooled_outputs = []
        with graph.as_default():
            self.embeddings = tf.Variable(E, dtype=tf.float32)
            self.train_inputs = tf.placeholder(tf.int32, shape=[1,max_len])
            embed = tf.nn.embedding_lookup(self.embeddings, self.train_inputs)
            print("embed after lookup",embed)
            embed = tf.expand_dims(embed, -1)
            for f_size in [1,3,5]:
                kernel_size = [f_size, 128, 1, 3]
                filter_weights = tf.Variable(tf.truncated_normal(kernel_size, stddev=0.1))
                bias_conv = tf.Variable(tf.zeros(3))
                conv = tf.nn.conv2d(embed,filter_weights,strides=[1, 1, 1, 1],padding="VALID")
                # Apply nonlinearity
                Relu_layer = tf.nn.relu(tf.nn.bias_add(conv, bias_conv))
                # Max-pooling over the outputs
                pooled = tf.nn.max_pool(Relu_layer,ksize=[1, max_len - f_size + 1, 1, 1],strides=[1, 1, 1, 1],padding='VALID')
                pooled_outputs.append(pooled)
            total_h_grams = 3 * 3
            concatenate_map_filters = tf.concat(pooled_outputs,3)
            print("Concatenated map filters", concatenate_map_filters)
            Cs_matrix = tf.reshape(concatenate_map_filters,[-1])
            print("Cs_matrx",Cs_matrix)
            self.init = tf.global_variables_initializer()
            
  
            
    def train(self, sess, trainFeats, max_len ):
        self.init.run()
        print("Initailized")
        c = 0
        for sent in trainFeats:
            feed_dict = {self.train_inputs.name: sent}
            #print("next")
            c +=1
        print("train finished")
        #print(c)
        #print(len(self.pooled_outputs))
        #return self.pooled_outputs
           


In [209]:
def init_cnn():
    E,unigram_prob,wrd2idx,word_counter,n_users = pickle.load(open('train_embeddings.pkl', 'rb'))
    print(E.shape)
    message_list = create_sent_corpus()
    print("Generating Traning Examples")
    trainFeats= genTrainExamples(message_list, max_len, wrd2idx)
    #print(trainFeats)
    print("Done.")

    # Build the graph model
    graph = tf.Graph()

    model = CNNModel(graph, E)

    with tf.Session(graph=graph) as sess:
        
        Cs_matrix = model.train(sess, trainFeats, max_len)
        
    

In [210]:
init_cnn()

(1034, 128)
Generating Traning Examples
Done.
embed after lookup Tensor("embedding_lookup:0", shape=(1, 10, 128), dtype=float32)
Concatenated map filters Tensor("concat:0", shape=(1, 1, 1, 9), dtype=float32)
Cs_matrx Tensor("Reshape:0", shape=(9,), dtype=float32)
Initailized
train finished
